In [3]:
pddl_description = """
;; PDDL-style planning problem: Attend Steelers Game

;; Predicates
;; At(x, l) — Object x is at location l
;; Has(ticket) — You possess a valid game ticket
;; CheckedIn(x) — Object x has passed stadium security
;; Fan(x) — Object x is a fan (you)
;; Location(l) — l is a valid location

Init(
  At(Juan, Home) ∧ Location(Home) ∧ Location(Stadium) ∧ Location(BoxOffice) ∧ Location(SecurityGate)
  ∧ Fan(Juan)
)

Goal(
  At(Juan, Stadium) ∧ Has(ticket) ∧ CheckedIn(Juan)
)

Action(BuyTicket(x, l),
  PRECOND: At(x, l) ∧ Fan(x) ∧ Location(l)
  EFFECT: Has(ticket)
)

Action(Travel(x, from, to),
  PRECOND: At(x, from) ∧ Location(from) ∧ Location(to)
  EFFECT: ¬At(x, from) ∧ At(x, to)
)

Action(CheckIn(x, l),
  PRECOND: At(x, l) ∧ Has(ticket) ∧ Location(l)
  EFFECT: CheckedIn(x)
)

;; Solvable Plan
[
  Travel(Juan, Home, BoxOffice),
  BuyTicket(Juan, BoxOffice),
  Travel(Juan, BoxOffice, SecurityGate),
  CheckIn(Juan, SecurityGate),
  Travel(Juan, SecurityGate, Stadium)
]
"""


In [4]:
# Steelers Game Planner (Classical STRIPS-style, Forward Search with A*)
# Complete corrected version — copy-paste this single cell into a Jupyter notebook and run.

from dataclasses import dataclass
from typing import List, Optional, Dict
import heapq

# ---------- Representation ----------

@dataclass(frozen=True)
class Action:
    name: str
    precond_pos: frozenset
    precond_neg: frozenset
    add: frozenset
    delete: frozenset

def make_fluent(pred: str, *args: str) -> str:
    return f"{pred}({', '.join(args)})"

# ---------- Domain Objects ----------

Juan = "Juan"
Home = "Home"
BoxOffice = "BoxOffice"
SecurityGate = "SecurityGate"
Stadium = "Stadium"

# ---------- Initial State and Goal ----------

init_state = frozenset({
    make_fluent("At", Juan, Home),
    make_fluent("Location", Home),
    make_fluent("Location", BoxOffice),
    make_fluent("Location", SecurityGate),
    make_fluent("Location", Stadium),
    make_fluent("Fan", Juan),
    # Closed-world assumption: Has(ticket) and CheckedIn(Juan) are false initially
})

goal = frozenset({
    make_fluent("At", Juan, Stadium),
    make_fluent("Has", "ticket"),
    make_fluent("CheckedIn", Juan),
})

# ---------- Action Schemas (Grounded for this tiny domain) ----------

def BuyTicket_at_BoxOffice(x: str) -> Action:
    # Ticket can ONLY be bought at BoxOffice
    return Action(
        name=f"BuyTicket({x}, {BoxOffice})",
        precond_pos=frozenset({
            make_fluent("At", x, BoxOffice),
            make_fluent("Fan", x),
            make_fluent("Location", BoxOffice),
        }),
        precond_neg=frozenset(),
        add=frozenset({ make_fluent("Has", "ticket") }),
        delete=frozenset(),
    )

def Travel_action(x: str, from_loc: str, to_loc: str) -> Action:
    # Travel to Stadium requires prior CheckIn.
    extra_preconds = set()
    if to_loc == Stadium:
        extra_preconds.add(make_fluent("CheckedIn", x))
    return Action(
        name=f"Travel({x}, {from_loc}, {to_loc})",
        precond_pos=frozenset({
            make_fluent("At", x, from_loc),
            make_fluent("Location", from_loc),
            make_fluent("Location", to_loc),
            *extra_preconds,
        }),
        precond_neg=frozenset(),
        add=frozenset({ make_fluent("At", x, to_loc) }),
        delete=frozenset({ make_fluent("At", x, from_loc) }),
    )

def CheckIn_at_SecurityGate(x: str) -> Action:
    # Check-in can ONLY occur at SecurityGate and requires a ticket
    return Action(
        name=f"CheckIn({x}, {SecurityGate})",
        precond_pos=frozenset({
            make_fluent("At", x, SecurityGate),
            make_fluent("Has", "ticket"),
            make_fluent("Location", SecurityGate),
        }),
        precond_neg=frozenset(),
        add=frozenset({ make_fluent("CheckedIn", x) }),
        delete=frozenset(),
    )

locations = [Home, BoxOffice, SecurityGate, Stadium]

# Ground actions, with realistic constraints:
ground_actions: List[Action] = []
ground_actions.append(BuyTicket_at_BoxOffice(Juan))

for frm in locations:
    for to in locations:
        if frm != to:
            ground_actions.append(Travel_action(Juan, frm, to))

ground_actions.append(CheckIn_at_SecurityGate(Juan))

# ---------- Planning Utilities ----------

def applicable(state: frozenset, action: Action) -> bool:
    return action.precond_pos.issubset(state) and state.isdisjoint(action.precond_neg)

def apply(state: frozenset, action: Action) -> frozenset:
    # RESULT(s, a) = (s − DEL(a)) ∪ ADD(a)
    return frozenset((state - action.delete) | action.add)

def goal_test(state: frozenset, goal: frozenset) -> bool:
    return goal.issubset(state)

def heuristic_unsatisfied_goals(state: frozenset, goal: frozenset) -> int:
    # Count of unsatisfied goal literals (simple, fast, admissible for positive conjunctive goals)
    return sum(1 for g in goal if g not in state)

# ---------- A* State-Space Search ----------

def astar(initial: frozenset,
          goal: frozenset,
          actions: List[Action],
          hfunc) -> Optional[List[Action]]:
    frontier = []
    g0 = 0
    h0 = hfunc(initial, goal)
    heapq.heappush(frontier, (g0 + h0, g0, initial, []))
    best_g: Dict[frozenset, int] = {initial: 0}

    while frontier:
        f, g, state, plan = heapq.heappop(frontier)
        if goal_test(state, goal):
            return plan
        for a in actions:
            if applicable(state, a):
                next_state = apply(state, a)
                g2 = g + 1
                if next_state not in best_g or g2 < best_g[next_state]:
                    best_g[next_state] = g2
                    h2 = hfunc(next_state, goal)
                    heapq.heappush(frontier, (g2 + h2, g2, next_state, plan + [a]))
    return None

# ---------- Solve, Display, and Validate ----------

plan = astar(init_state, goal, ground_actions, heuristic_unsatisfied_goals)

if plan is None:
    print("No plan found.")
else:
    print("Plan found with", len(plan), "steps:")
    for i, a in enumerate(plan, 1):
        print(f"{i}. {a.name}")

def simulate_plan(initial: frozenset, plan: List[Action]) -> frozenset:
    s = initial
    for a in plan:
        if not applicable(s, a):
            raise RuntimeError(f"Action not applicable: {a.name}")
        s = apply(s, a)
    return s

final_state = simulate_plan(init_state, plan or [])

print("\nFinal state contains goal literals?")
for g in goal:
    print(f"- {g}: {'YES' if g in final_state else 'NO'}")

print("\nFinal state sample fluents (up to 12):")
for f in sorted(list(final_state))[:12]:
    print(f"  {f}")



Plan found with 5 steps:
1. Travel(Juan, Home, BoxOffice)
2. BuyTicket(Juan, BoxOffice)
3. Travel(Juan, BoxOffice, SecurityGate)
4. CheckIn(Juan, SecurityGate)
5. Travel(Juan, SecurityGate, Stadium)

Final state contains goal literals?
- Has(ticket): YES
- CheckedIn(Juan): YES
- At(Juan, Stadium): YES

Final state sample fluents (up to 12):
  At(Juan, Stadium)
  CheckedIn(Juan)
  Fan(Juan)
  Has(ticket)
  Location(BoxOffice)
  Location(Home)
  Location(SecurityGate)
  Location(Stadium)
